In [1]:
# ! pip install transformers
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW

import torchsummary as summary
from tqdm import tqdm

from torch import nn
import torch.nn.functional as F
from transformers import BertModel
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
import torch

import pandas as pd
import numpy as np
import os
import base64
import gc
import glob, os

from torch.optim import Adam
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

import json
from general_functions import *

C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
'''
def create_onedrive_directdownload(onedrive_link):
    data_bytes64 = base64.b64encode(bytes(onedrive_link, 'utf-8'))
    data_bytes64_String = data_bytes64.decode('utf-8').replace('/','_').replace('+','-').rstrip("=")
    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"
    return resultUrl
'''

'\ndef create_onedrive_directdownload(onedrive_link):\n    data_bytes64 = base64.b64encode(bytes(onedrive_link, \'utf-8\'))\n    data_bytes64_String = data_bytes64.decode(\'utf-8\').replace(\'/\',\'_\').replace(\'+\',\'-\').rstrip("=")\n    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"\n    return resultUrl\n'

In [3]:
'''
onedrive_link='https://1drv.ms/u/s!AoiE7xOoBAsngsgx2HKIw1RxUnATTg?e=XsxZGD'
onedrive_direct_link=create_onedrive_directdownload(onedrive_link)
onedrive_direct_link
df=pd.read_csv(onedrive_direct_link)
'''

"\nonedrive_link='https://1drv.ms/u/s!AoiE7xOoBAsngsgx2HKIw1RxUnATTg?e=XsxZGD'\nonedrive_direct_link=create_onedrive_directdownload(onedrive_link)\nonedrive_direct_link\ndf=pd.read_csv(onedrive_direct_link)\n"

In [4]:
# Import data
# Access aws credentials from json file
with open("../aws_credentials.json", 'r') as file:
    aws_creds_json = json.load(file)
# Specify s3 bucket
bucket = "fs-reghub-news-analysis"

# Connect to aws and dowload the files
aws = awsOps(aws_creds_json)
df = aws.get_df(bucket=bucket, file="data_rule_labels_v1.csv")
df_categories = aws.get_df(bucket=bucket, file="rule_labels_v1.csv")

In [5]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df=df[['news_content','rule_labels_comb']]
df=df[~df['rule_labels_comb'].isna()]
df=df[(df['rule_labels_comb'].apply(len)!=2)]
df = df.reset_index()
del df['index']
df

,news_content,rule_labels_comb
0,Berenberg Bank analysts have provided a buy ra...,['market']
1,"The article states that Berenberg, a German in...",['market']
2,"In their analysis on October 30, 2023, experts...",['market']
3,The private bank Berenberg has upgraded its ra...,"['papers', 'market']"
4,In a research note published by Sebastian Bray...,"['sanctions', 'papers', 'market']"
...,...,...
5517,INTERVIEW Interview with Les Échos Interview w...,"['legal', 'statements', 'guidelines']"
5518,UBS's latest Investor Watch report reveals tha...,"['reports', 'personnel']"
5519,SNB erwartet für 2021 Jahresgewinn von rund 26...,"['legal', 'reports', 'market']"
5520,0:00 News A cryptocurrency exchange in Hong Ko...,"['legal', 'reports', 'guidelines', 'press']"


In [6]:
from ast import literal_eval
df['rule_labels_comb'] = df['rule_labels_comb'].apply(literal_eval)
df['target']='[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]'
df['target'] = df['target'].apply(literal_eval)

df['target'].iloc[0][0]

0.0

In [7]:

for index, value in df['rule_labels_comb'].iteritems():
    class_dic={'legal':0,'sanctions':1,'papers':2,'reports':3,'statements':4,'guidelines':5,'press':6,'personnel':7,'market':8}
    val=[class_dic.get(i, i) for i in value]
    df['rule_labels_comb'].iloc[index]=val
    y=0
    for x in df['rule_labels_comb'].iloc[index]:
        df['target'].iloc[index][x]=1.0
        y=y+1
df['rule_labels_comb']

0                [8]
1                [8]
2                [8]
3             [2, 8]
4          [1, 2, 8]
            ...     
5517       [0, 4, 5]
5518          [3, 7]
5519       [0, 3, 8]
5520    [0, 3, 5, 6]
5521    [0, 3, 4, 6]
Name: rule_labels_comb, Length: 5522, dtype: object

In [8]:
df['target']

0       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
1       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
2       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
3       [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
4       [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
                            ...                      
5517    [1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]
5518    [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
5519    [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]
5520    [1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0]
5521    [1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]
Name: target, Length: 5522, dtype: object

In [9]:
# import data from gdrive
'''
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/')


df=pd.read_csv('BERT_data.csv')
df=df[~(df['content']=='nan')]
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\nos.chdir('/content/drive/My Drive/')\n\n\ndf=pd.read_csv('BERT_data.csv')\ndf=df[~(df['content']=='nan')]\ndf['content']=df['content'].astype(str)\ndf['subject']=df['subject'].astype(str)\n"

In [10]:
# change range of labels, minimum should be zero
df['rule_labels_comb']=df['rule_labels_comb'].astype(str)
df

,news_content,rule_labels_comb,target
0,Berenberg Bank analysts have provided a buy ra...,[8],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
1,"The article states that Berenberg, a German in...",[8],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
2,"In their analysis on October 30, 2023, experts...",[8],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
3,The private bank Berenberg has upgraded its ra...,"[2, 8]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
4,In a research note published by Sebastian Bray...,"[1, 2, 8]","[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
...,...,...,...
5517,INTERVIEW Interview with Les Échos Interview w...,"[0, 4, 5]","[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]"
5518,UBS's latest Investor Watch report reveals tha...,"[3, 7]","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
5519,SNB erwartet für 2021 Jahresgewinn von rund 26...,"[0, 3, 8]","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
5520,0:00 News A cryptocurrency exchange in Hong Ko...,"[0, 3, 5, 6]","[1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0]"


In [11]:
# additional filtering to balance classes
'''
df_3=df[(df['result']==3) & (df['content'].str.len()<350)]
df=df[~(df['result']==3)]
df=pd.concat([df,df_3])
df
'''

"\ndf_3=df[(df['result']==3) & (df['content'].str.len()<350)]\ndf=df[~(df['result']==3)]\ndf=pd.concat([df,df_3])\ndf\n"

In [12]:
# import BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Dataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.labels=df['target']
        self.text=[tokenizer(text,padding='max_length',truncation=True,return_tensors="pt") for text in df['news_content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [13]:
# train test split
X_train, X_test= train_test_split(df[['news_content','target']], test_size=0.25)

In [14]:
df_train=X_train
df_val=X_test
df_test=0

In [15]:
df_train

,news_content,target
1269,The article discusses the planned downsizing o...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
22,In its quarterly report for the third quarter ...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1843,The article discusses the recent rally in the ...,"[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
1897,The DAX is expected to open higher at the star...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
366,The article discusses the connection between D...,"[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
...,...,...
2447,The German stock market index (DAX) closed sli...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
3968,Separately Berenberg Bank cut their price targ...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1741,The article discusses the review of the crimin...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3043,Asset Management Investment Bank Wealth Mana...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"


In [16]:
# BERT classifier architecture, with 7 output classes
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 9)
        torch.nn.init.kaiming_uniform_(self.linear.weight, nonlinearity='relu')
        self.relu = nn.ReLU()
        
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


In [17]:
# change runtype to GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [18]:
# hyperparameters
EPOCHS = 100
model = BertClassifier()
LR = 0.00001

In [19]:
df_train=df_train.reset_index()
df_train.drop('index',axis=1,inplace=True)

df_val=df_val.reset_index()
df_val.drop('index',axis=1,inplace=True)
df_val

,news_content,target
0,Anita Dushyanth is a Healthcare equity researc...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
1,Hinter unseren beiden heutigen Protagonisten D...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
2,Die Experten der RBC Capital Markets bewerten ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
3,Flossbach von Storch is one of Europe's larges...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,The use of Artificial Intelligence (AI) in hea...,"[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
...,...,...
1376,"Oddo BHF, a financial services group, has main...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
1377,"HSBC Trinkaus & Burkhardt, a financial analyst...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
1378,Kaum verändert im Vergleich zu der letzten Not...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
1379,Am deutschen Aktienmarkt ist die Aktie der Com...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"


In [20]:
# display BERT layers
n=0
for x in model.state_dict():
    n=n+1
    print(x)
n

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

201

In [21]:
# freeze first 8 layers 
n=0
for param in model.parameters():
    n=n+1
    param.requires_grad = False
    if n==(201-68):
        break

In [22]:
# change datatypes of input data
df_train['news_content']=df_train['news_content'].astype(str)

df_val['news_content']=df_val['news_content'].astype(str)
df_train['news_content'].iloc[0]

'The article discusses the planned downsizing of Credit Suisse, with a particular impact on its investment banking division. The staff reduction will be carried out in multiple stages.'

In [24]:
torch.cuda.empty_cache()
gc.collect()

0

In [25]:
def train(model, train_data, val_data, learning_rate, epochs):
    
    stop_criteria=0
    
    train, val = Dataset(train_data), Dataset(val_data)
    
    # mini batching
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=32)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=32)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)
    
    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()
            
    plot_val_acc=[]
    plot_train_acc=[]
    plot_epoch=[]
    plot_train_loss=[]
    plot_val_loss=[]

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            true_acc=0
            n=0
            with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch_num + 1}/{epochs}', unit='item',position=0,leave=True) as p_bar:
                for train_input, train_label in train_dataloader:
                    train_label = train_label.to(device) # to cuda GPU
                    mask = train_input['attention_mask'].to(device) # attention mask
                    input_id = train_input['input_ids'].squeeze(1).to(device)

                    l1_loss=0

                    # for L1 regularization
                    a=0
                    reg_loss = 0
                    for param in model.parameters():
                        a=a+1
                        if a >=201-68:
                            reg_loss += torch.norm(param, 1) 
                    '''
                    factor = 0.00001 #lambda for L1 regularization
                    l1_loss=factor * reg_loss # L1 loss
                    '''
                    # model output
                    output = model(input_id, mask)


                    # loss value
                    batch_loss = criterion(output, train_label) + l1_loss
                    total_loss_train += batch_loss.item() 

                    # train accuracy 
                    output=output.squeeze()
                    train_label=train_label.squeeze()
                    # print(output)
                    # print(train_label)
                    # print(torch.gather(train_label, 1, torch.argmax(output,dim=1).view(-1, 1)))
                    acc = torch.gather(train_label, 1, torch.argmax(output,dim=1).view(-1, 1)).sum().item()
                    total_acc_train += acc
                    
                    # true accuracy
                    true_acc_batch=0
                    for (x,y) in zip(torch.round(F.sigmoid(output)*10)/10,train_label): # ceil or round
                        x=torch.tensor(x)
                        y=torch.tensor(y)
                        # print(x)
                        # print(y)
                        x[x<1]=0
                        true_acc=true_acc+torch.sum(x==y).item()
                        true_acc_batch=true_acc_batch+torch.sum(x==y).item()


                    # backpropogation
                    model.zero_grad()
                    batch_loss.backward()
                    optimizer.step()
                    p_bar.set_postfix(loss=batch_loss.item() / len(train_input['input_ids']), acc=acc / len(train_input['input_ids']), true_acc=true_acc_batch / (len(train_input['input_ids'])*9))
                    p_bar.update()
                p_bar.set_postfix({'Epoch': epoch_num + 1})
            # print(output)
            # print(train_label)

            total_acc_val = 0
            total_loss_val = 0
            true_val_acc=0
            
            # for validation accuracy
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)
                    
                    # validation output
                    output = model(input_id, mask)
                    
                    # validation loss value
                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                
                    # validation accuracy
                    output=output.squeeze()
                    val_label=val_label.squeeze()
                    acc = torch.gather(val_label, 1, torch.argmax(output,dim=1).view(-1, 1)).sum().item()
                    total_acc_val += acc
                    
                    # true accuracy
                    for (x,y) in zip(torch.round(F.sigmoid(output)*10)/10,val_label): # ceil or round
                        x=torch.tensor(x)
                        y=torch.tensor(y)
                        x[x<1]=0
                        true_val_acc=true_val_acc+torch.sum(x==y).item()
            
            print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f} \
            | True Train Accuracy: {true_acc / (len(train_data)*9): .3f} \
            | True Val Accuracy: {true_val_acc / (len(val_data)*9): .3f}',end='\r')
            
            plot_train_acc.append(total_acc_train / len(train_data))
            plot_val_acc.append(total_acc_val / len(val_data))
            plot_epoch.append(epoch_num + 1)
            plot_train_loss.append(total_loss_train / (len(train_data)*9))
            plot_val_loss.append(total_loss_val / (len(val_data)*9))
            
            # model checkpoint
            checkpoint = {
            'epoch': epoch_num+1,
            'model_state_dict': model.state_dict(),
            }
            
            torch.save(checkpoint, f'BERT_checkpoint_epoch{epoch_num+1}.pth')
            
            try:
                if plot_val_loss[-1]>plot_val_loss[-2] and plot_val_loss[-2]>plot_val_loss[-3]:
                    break
            except:
                pass
    
        
    fig, ax = fig,ax= plt.subplots(1, 2, figsize=(10, 7))
    fig.suptitle('Accuracy and Loss')
    
    
    
    ax[0].plot(plot_epoch,plot_val_acc, label='Validation Accuracy',color='orange')
    ax[0].plot(plot_epoch,plot_train_acc, label='Training Accuracy',color='red')
    ax[0].legend()
    ax[0].grid(True)
    
    ax[1].plot(plot_epoch,plot_train_loss, label=' Training Loss',color='blue')
    ax[1].plot(plot_epoch,plot_val_loss, label=' Validation Loss',color='yellow')
    ax[1].legend()
    ax[1].grid(True)
    
    plt.show()
    
    # retrieve best checkpoint model
    checkpoint = torch.load(f'BERT_checkpoint_epoch{epoch_num+1-2}.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    
    
    # save best model
    torch.save(model, 'BERT_classifier.pth')

    # save model to aws S3 cloud
    with open("../aws_credentials.json", 'r') as file:
        aws_creds_json = json.load(file)
    bucket = "fs-reghub-news-analysis"
    aws = awsOps(aws_creds_json)
    aws.upload_file(bucket=bucket, name):
    
    # delete remaining model checkpoints
    for f in glob.glob("BERT_checkpoint*.pth"):
        os.remove(f)  
        
        
train(model, df_train, df_val, LR, EPOCHS)

Epoch 1/100: 100%|████████████████████████████████████████████████████████| 130/130 [08:08<00:00,  3.76s/item, Epoch=1]


Epoch 2/100:   9%|██▍                       | 12/130 [00:49<08:11,  4.16s/item, acc=0.719, loss=0.0775, true_acc=0.851]


KeyboardInterrupt: 

In [ ]:
with open("../aws_credentials.json", 'r') as file:
        aws_creds_json = json.load(file)
bucket = "fs-reghub-news-analysis"
aws = awsOps(aws_creds_json)



aws.upload_file(bucket=bucket,path='BERT_classifier.pth', name='BERT_classifier.pth')

SyntaxError: EOL while scanning string literal (3406149190.py, line 2)